In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re

# Import CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Username and Password for MongDB database
username = "aacuser"
password = "Jeremy1990"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Logo for Grazioso Salvare
image_filename = 'Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div(
    # Set style for logo and page title (centered)
    style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center', 'margin': '0', 'padding': '0'},
    children=[
        html.A(
            children=[
                html.Img(
                    src=f'data:image/png;base64, {encoded_image.decode()}',
                    style={'height': '260px', 'width': '260px'}
                )
            ],
            # Attach link to logo that leads to snhu.edu
            href='https://www.snhu.edu',
            target="_blank",
            style={'display': 'block', 'margin': '0 auto'}
        ),
        html.H1(
            # Dashboard title
            'Jeremy Snow\'s SNHU CS340 Dashboard',
            style={'textAlign': 'center', 'fontSize': '25px', 'margin': '10px 0'}
        ),
        html.Hr(),
        html.Div([
            # Interactive filtering options
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'All', 'value': 'All'},
                    {'label': 'Water Rescue', 'value': 'Water'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'}
                ],
                value='All',
                labelStyle={'display': 'inline-block'}
            ) 
        ]),
        html.Hr(),
        html.Div(
            style={'width': '100%', 'overflowX': 'auto'},
            children=[
                # Datatable with added features to increase user-friendliness
                dash_table.DataTable(
                    id='datatable-id',
                    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                    data=df.to_dict('records'),
                    # Enable native filtering
                    filter_action="native",
                    # Enable native sorting
                    sort_action="native",
                    # Enable multi-column sorting
                    sort_mode="multi",
                    # Enable single mode of row selection
                    row_selectable="single",
                    # Select the first row by default
                    selected_rows=[0],
                    # Enable native paging
                    page_action="native",
                    # Set initial page to 0
                    page_current=0,
                    # Show 10 rows at a time
                    page_size=10 
                    )
                ]
            ),
        html.Br(),
        html.Hr(),
        # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
        html.Div(
            
            style={'display': 'flex', 'width': '100%'}, 
            children=[
                html.Div(id='map-id', className='col s12 m6', style={'flex': '1', 'padding': '10px'}),
                html.Div(id='graph-id', className='col s12 m6', style={'flex': '1', 'padding': '10px'})
            ]
        )
    ]
)

#############################################
# Interaction Between Components / Controller
#############################################

# Selects the first visible row regardless of native filtering
@app.callback(
    Output('datatable-id', 'selected_rows'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_indices')]
)
def update_selected_rows(visible_data, visible_indices):
    # Ensure that the first row is selected if there is visible data
    if visible_data and visible_indices:
        # Get the first index from the derived_virtual_indices
        first_visible_index = visible_indices[0]
        return [first_visible_index]
    else:
        # If no data is visible, clear the selection
        return []

# Updates the dashboard based on selected filters
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):

    # Define regex patterns for filtering in a dictionary
    # Helps capture variations of each breed
    regex_patterns = {
        'Water': [re.compile(".*lab.*", re.IGNORECASE),
                  re.compile(".*chesa.*", re.IGNORECASE),
                  re.compile(".*newf.*", re.IGNORECASE)],
        'Mountain': [re.compile(".*german she.*", re.IGNORECASE),
                     re.compile(".*alaskan mala.*", re.IGNORECASE),
                     re.compile(".*old english s.*", re.IGNORECASE),
                     re.compile(".*husk.*", re.IGNORECASE),
                     re.compile(".*rott.*", re.IGNORECASE)],
        'Disaster': [re.compile(".*dober.*", re.IGNORECASE),
                     re.compile(".*german she.*", re.IGNORECASE),
                     re.compile(".*golden.*", re.IGNORECASE),
                     re.compile(".*blood.*", re.IGNORECASE),
                     re.compile(".*rott.*", re.IGNORECASE)]
    }

    # Initialize query
    query = {}

    # Set filter criteria based on filter type
    if filter_type == 'All':
        # Show all animals
        query = {}
    elif filter_type in regex_patterns:
        # Filter based on selected filter
        patterns = regex_patterns[filter_type]
        query = {
            '$or': [{'breed': {'$regex': pattern}} for pattern in patterns],
            "sex_upon_outcome": "Intact Male" if filter_type in ['Mountain', 'Disaster'] else "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0} if filter_type in ['Water', 'Mountain'] else {"$gte": 20.0, "$lte": 300.0}
        }
    else:
        raise Exception("Unknown filter")

    # Obtain query results
    results = shelter.read(query)

    # Convert results to DataFrame
    df = pd.DataFrame(results)

    # Drop _id column
    df.drop(columns=['_id'],inplace=True)
    data=df.to_dict('records')
    
    return data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    # Handle the case where there is no data (e.g., return an empty map or a placeholder)
    if viewData is None or len(viewData) == 0:
        return []

    # Build DataFrame from the viewData
    df = pd.DataFrame.from_dict(viewData)
    
    # Count the occurrences of each breed
    breed_counts = df['breed'].value_counts()
    
    # Set the threshold for what counts as a small slice
    threshold = 0.01  # 1%
    
    # Identify breeds that will be included in the "Other" category
    total_count = breed_counts.sum()
    small_breeds = breed_counts[breed_counts / total_count < threshold].index
    
    # Aggregate the small breeds into an "Other" category
    df['breed'] = df['breed'].apply(lambda x: x if x not in small_breeds else 'Other')
    
    # Count the occurrences of each breed after aggregation
    aggregated_breed_counts = df['breed'].value_counts()
    
    # Create the pie chart
    fig = px.pie(aggregated_breed_counts, names=aggregated_breed_counts.index, values=aggregated_breed_counts.values, title='Preferred Animals')

    # Customize the tooltip format
    fig.update_traces(
        hovertemplate="<b>Breed:</b> %{label}<br><b>Count:</b> %{value}<extra></extra>"
    )

    # Customize the layout for the title
    fig.update_layout(
        title={
            'text': 'Preferred Animals',  # Title text
            'font': {
                'size': 24,  # Title font size
                'family': "Arial, sans-serif",  # Title font family
            }
        }
    )
    
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'derived_virtual_selected_rows'),
    Input('datatable-id', 'selected_rows')]
)
def update_styles(virtual_selected_rows, selected_rows):
    # Before filtering
    if not virtual_selected_rows:
        return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]
    # After filtering
    else:
        return [{
            'if': { 'row_index': i },
            'background_color': '#D2F3FF'
        } for i in virtual_selected_rows]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    # Handle the case where there is no data (e.g., return an empty map or a placeholder)
    if not viewData or not index or index[0] >= len(viewData):
        return []

    # Build a DataFrame from the viewData
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            # center is set to currently selected row's location data
            center=[dff.iloc[row, 13], dff.iloc[row, 14]], 
            zoom=10, 
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                dl.Marker(
                    position=[dff.iloc[row,13],dff.iloc[row,14]], 
                    children=[
                        # Tooltip that displays the breed of the animal upon hovering over the pinned location
                        dl.Tooltip(dff.iloc[row,4]),
                        dl.Popup([
                            html.Div([
                                # Custom popup that displays animal's information
                                html.H3("Animal Information", style={'color': '#007bff'}),
                                html.H4(dff.iloc[row, 4], style={'fontWeight': 'bold'}),
                                html.P(f"Name: {dff.iloc[row, 9]}", style={'margin': '5px 0', 'fontSize': '14px'}),
                                html.P(f"Age: {int(dff.iloc[row, 15])} weeks", style={'margin': '5px 0', 'fontSize': '14px'}),
                                html.P(f"Location: [{dff.iloc[row, 13]}, {dff.iloc[row, 14]}]", style={'margin': '5px 0', 'fontSize': '14px'}),
                                html.P(f"Outcome: {dff.iloc[row, 11]}", style={'margin': '5px 0', 'fontSize': '14px'})
                            ], style={'padding': '10px', 'fontFamily': 'Arial, sans-serif', 'textAlign': 'center'})
                        ])
                    ]
                )
            ]
        )
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/
